Renato Dias Ferreira Campos

<h1>Problema do Caixeiro Viajante</h1>

Suponha que um caixeiro viajante tenha que visitar \( n \) cidades diferentes, iniciando e encerrando sua viagem na **primeira cidade**. Suponha, também, que:

1. **A ordem das visitas não importa.**
2. De qualquer cidade, o caixeiro pode ir diretamente para qualquer outra.

O objetivo do problema do Caixeiro Viajante é **descobrir a rota que minimiza a distância total da viagem**, considerando que o ponto de partida e o ponto final são a mesma cidade.


<h2/>Importando bibliotecas</h2>

In [ ]:
import numpy as np
import random

### Declarando número de cidades e gerando distâncias de maneira aleatória

In [ ]:
#Número de cidades
n = 6

distancias = np.zeros((n, n), dtype = int)
for i in range(n):
  for j in range(i):
    distancias[i][j] = random.randint(1,100)
    distancias[j][i] = distancias[i][j]

print(distancias)

[[ 0 50 32 69 18 41]
 [50  0 69 11 30 75]
 [32 69  0 16 84 60]
 [69 11 16  0 88 63]
 [18 30 84 88  0 97]
 [41 75 60 63 97  0]]


### Inicialização dos indvíduos

In [ ]:
#Escolhemos um número pequeno de indivíduos pois existem poucas combinações possíveis
#Dessa maneira, aumentamos as chances do caminho mais curto ainda não estar presente na nossa população inicial
#Garantindo que nosso algoritmo genético possa ser observado com resultados

num_individuos = 10
viagens = n + 1
populacao_inicial = np.zeros((num_individuos, viagens), dtype=int)

#Garantindo que a cidade 0 seja o início e fim da viagem
for i in range(num_individuos):
  populacao_inicial[i][0] = 0
  populacao_inicial[i][n] = 0

  populacao_inicial[i][1:n] = random.sample(range(1, n), n - 1)

for i in range(num_individuos):
  print(populacao_inicial[i])

[0 5 3 1 4 2 0]
[0 1 4 3 2 5 0]
[0 4 5 2 3 1 0]
[0 3 1 5 2 4 0]
[0 1 3 4 2 5 0]
[0 4 3 5 2 1 0]
[0 3 5 4 1 2 0]
[0 2 1 3 5 4 0]
[0 5 4 1 3 2 0]
[0 3 5 2 1 4 0]


### Definindo a função fitness
Ela será calculada por meio da soma das distâncias, usando linha X coluna da matriz



In [ ]:
def fitness(ind):

  f = 0

  #Soma das distâncias
  for i in range(1, len(ind)):
    f = f + distancias[ind[i-1]][ind[i]]

  return f


def retorna_fitness(populacao):

  fit = []

  #Criação da list de fitnesses
  for i in range(len(populacao)):
    f = fitness(populacao[i])
    fit.append(f)

  return fit


### Seleção por torneio

In [ ]:
def selecao_torneio(fitnesses):

    ind1 = -1
    ind2 = -1
    while ind1 == ind2:
        # Torneio 1
        sorteados = random.sample(range(0, len(fitnesses)), 2)
        if fitnesses[sorteados[0]] < fitnesses[sorteados[1]]:
            ind1 = sorteados[0]
        else:
            ind1 = sorteados[1]

        # Torneio 2
        sorteados = random.sample(range(0, len(fitnesses)), 2)
        if fitnesses[sorteados[0]] < fitnesses[sorteados[1]]:
            ind2 = sorteados[0]
        else:
            ind2 = sorteados[1]

    return ind1, ind2

### Cruzamento Order Crossover

Usado para garantir as regras das viagens, ou seja, deve-se passar por todas as cidades e não deve haver repetição.

In [ ]:
def cruzamento(ids, populacao):
    pai1 = populacao[ids[0]]
    pai2 = populacao[ids[1]]

    #Criação dos pontos de começo e fim
    start, end = sorted(random.sample(range(1, len(pai1) - 1), 2))

    filho = [0] * len(pai1)
    #O filho recebe os elementos do pai1 de start até end
    filho[start:end] = pai1[start:end]

    pos = end
    #O filho recebe os elementos do pai2 que ainda não estão presentes em filho
    for gene in pai2[1:-1]:
        if gene not in filho:
            if pos >= len(pai1) - 1:
                pos = 1
            filho[pos] = gene
            pos += 1

    filho[0], filho[-1] = 0, 0
    return filho

### Exemplificação do Cruzamento Order Crossover

In [ ]:
pai_exemplo_1 = [0, 2, 1, 4, 5, 3, 0]
pai_exemplo_2 = [0, 2, 3, 1, 4, 5, 0]

start, end = sorted(random.sample(range(1, len(pai_exemplo_1) - 1), 2))

filho = [0] * len(pai_exemplo_1)
filho[start:end] = pai_exemplo_1[start:end]

print(f"Start: {start}\nEnd: {end}")
print(f"Pai1: {pai_exemplo_1}\nPai2: {pai_exemplo_2}")

pos = end
for gene in pai_exemplo_2[1:-1]:
    if gene not in filho:
        if pos >= len(pai_exemplo_1) - 1:
            pos = 1
        print(filho)
        filho[pos] = gene
        pos += 1

filho[0], filho[-1] = 0, 0

print(f"Filho: {filho}")

Start: 1
End: 3
Pai1: [0, 2, 1, 4, 5, 3, 0]
Pai2: [0, 2, 3, 1, 4, 5, 0]
[0, 2, 1, 0, 0, 0, 0]
[0, 2, 1, 3, 0, 0, 0]
[0, 2, 1, 3, 4, 0, 0]
Filho: [0, 2, 1, 3, 4, 5, 0]


### Elitismo

In [ ]:
def elitismo(fitnesses):
    id_melhor = fitnesses.index(min(fitnesses))

    fitnesses.pop(id_melhor)
    id_melhor2 = fitnesses.index(min(fitnesses))

    return id_melhor, id_melhor2


### Criação das Permutações

Visto que temos 120 possibilidades de viagens, podemos enumerá-las e verificar a menor distância possível. Assim, podemos comparar com o resultado do nosso algoritmo genético

In [ ]:
import itertools

cidades = [1, 2, 3, 4, 5]
permutacoes = list(itertools.permutations(cidades))

matriz = []

#Criação de todas as viagens
for p in permutacoes:
    p = [0] + list(p) + [0]
    matriz.append(list(p))

In [ ]:
fitnesses = retorna_fitness(matriz)
i = 1

#Retorna todas as viagens posssíveis e suas respectivas distâncias
for viagem, distancia in zip(matriz, fitnesses):
    print(f"Viagem {i}: {viagem}: Distância: {distancia}")
    i += 1

Viagem 1: [0, 1, 2, 3, 4, 5, 0]: Distância: 361
Viagem 2: [0, 1, 2, 3, 5, 4, 0]: Distância: 313
Viagem 3: [0, 1, 2, 4, 3, 5, 0]: Distância: 395
Viagem 4: [0, 1, 2, 4, 5, 3, 0]: Distância: 432
Viagem 5: [0, 1, 2, 5, 3, 4, 0]: Distância: 348
Viagem 6: [0, 1, 2, 5, 4, 3, 0]: Distância: 433
Viagem 7: [0, 1, 3, 2, 4, 5, 0]: Distância: 299
Viagem 8: [0, 1, 3, 2, 5, 4, 0]: Distância: 252
Viagem 9: [0, 1, 3, 4, 2, 5, 0]: Distância: 334
Viagem 10: [0, 1, 3, 4, 5, 2, 0]: Distância: 338
Viagem 11: [0, 1, 3, 5, 2, 4, 0]: Distância: 286
Viagem 12: [0, 1, 3, 5, 4, 2, 0]: Distância: 337
Viagem 13: [0, 1, 4, 2, 3, 5, 0]: Distância: 284
Viagem 14: [0, 1, 4, 2, 5, 3, 0]: Distância: 356
Viagem 15: [0, 1, 4, 3, 2, 5, 0]: Distância: 285
Viagem 16: [0, 1, 4, 3, 5, 2, 0]: Distância: 323
Viagem 17: [0, 1, 4, 5, 2, 3, 0]: Distância: 322
Viagem 18: [0, 1, 4, 5, 3, 2, 0]: Distância: 288
Viagem 19: [0, 1, 5, 2, 3, 4, 0]: Distância: 307
Viagem 20: [0, 1, 5, 2, 4, 3, 0]: Distância: 426
Viagem 21: [0, 1, 5, 3, 2, 4,

In [ ]:
# Encontrar a distância mínima
distancia_minima = min(fitnesses)

# Filtrar todas as rotas com a mesma distância mínima
viagens_minimas = []
for i, distancia in enumerate(fitnesses):
    if distancia == distancia_minima:
        viagens_minimas.append((i + 1, matriz[i]))

# Imprimir os resultados
print(f"Viagens com a menor distância ({distancia_minima}):")
for viagem in viagens_minimas:
    indice, rota = viagem
    print(f"Viagem {indice}: {rota}")


Viagens com a menor distância (176):
Viagem 75: [0, 4, 1, 3, 2, 5, 0]
Viagem 105: [0, 5, 2, 3, 1, 4, 0]


### Mutação

A mutação é feita invertendo a posição de um elemento de índice idx1 com um elemento de índice idx2.

In [ ]:
def mutacao(filho, taxa):
    if random.random() < taxa:
        idx1, idx2 = random.sample(range(1, len(filho) - 1), 2)
        filho[idx1], filho[idx2] = filho[idx2], filho[idx1]
    return filho

### Programa principal do Algoritmo Evolutivo

In [ ]:
num_geracoes = 100
for it in range(num_geracoes):
    nova_populacao = np.zeros((num_individuos, viagens), dtype=int)

    # Fitness da população
    fit = retorna_fitness(populacao_inicial)

    # Melhor indivíduo
    id_melhor = fit.index(min(fit))
    print(f"Melhor rota da geração {it + 1}: {populacao_inicial[id_melhor]} com custo {fit[id_melhor]}")
    print(f"Viagens com a menor distância ({distancia_minima}):")
    for viagem in viagens_minimas:
        indice, rota = viagem
        print(f"Viagem {indice}: {rota}")
    print()

    # Elitismo
    elite = elitismo(fit)
    nova_populacao[0] = populacao_inicial[elite[0]]
    nova_populacao[1] = populacao_inicial[elite[1]]

    # Geração dos filhos
    num_filhos = 2
    while num_filhos < num_individuos:
        vencedores = selecao_torneio(fit)
        filho = cruzamento(vencedores, populacao_inicial)
        filho = mutacao(filho, 0.5)
        nova_populacao[num_filhos] = filho
        num_filhos += 1

    # Atualizar população
    populacao_inicial = nova_populacao.copy()

Melhor rota da geração 1: [0 5 4 1 3 2 0] com custo 227
Viagens com a menor distância (176):
Viagem 75: [0, 4, 1, 3, 2, 5, 0]
Viagem 105: [0, 5, 2, 3, 1, 4, 0]

Melhor rota da geração 2: [0 5 4 1 3 2 0] com custo 227
Viagens com a menor distância (176):
Viagem 75: [0, 4, 1, 3, 2, 5, 0]
Viagem 105: [0, 5, 2, 3, 1, 4, 0]

Melhor rota da geração 3: [0 5 4 1 3 2 0] com custo 227
Viagens com a menor distância (176):
Viagem 75: [0, 4, 1, 3, 2, 5, 0]
Viagem 105: [0, 5, 2, 3, 1, 4, 0]

Melhor rota da geração 4: [0 5 4 1 3 2 0] com custo 227
Viagens com a menor distância (176):
Viagem 75: [0, 4, 1, 3, 2, 5, 0]
Viagem 105: [0, 5, 2, 3, 1, 4, 0]

Melhor rota da geração 5: [0 5 4 1 3 2 0] com custo 227
Viagens com a menor distância (176):
Viagem 75: [0, 4, 1, 3, 2, 5, 0]
Viagem 105: [0, 5, 2, 3, 1, 4, 0]

Melhor rota da geração 6: [0 4 1 3 5 2 0] com custo 214
Viagens com a menor distância (176):
Viagem 75: [0, 4, 1, 3, 2, 5, 0]
Viagem 105: [0, 5, 2, 3, 1, 4, 0]

Melhor rota da geração 7: [0 4 1 3